In [3]:
import gym
import random

random.seed(1234)

streets = gym.make("Taxi-v3").env #New versions keep getting released; if -v3 doesn't work, try -v2 or -v4
streets.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



First we import the open source library — gym and start working with the taxi environment. Here we used version 3 but new versions keeps getting released, so if it doesn’t work then try v2 or v4.

We can use streets.render() to output a visualization of our environment, with our cute little taxi.

In [23]:
initial_state = streets.encode(2, 3, 2, 0) # 1,0,2,0

streets.s = initial_state

streets.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)


We will set a initial state for our problem, the agent start at row index 2 and column index 3, we need to pick up customer from Yellow spot and drop him off at Red spot.

In [5]:
import numpy as np

q_table = np.zeros([streets.observation_space.n, streets.action_space.n])
# a 2D array that represent every possible state and action in the virtual space and initialize all of them to 0
learning_rate = 0.1
discount_factor = 0.6
exploration = 0.1
epochs = 10000

for taxi_run in range(epochs):
    state = streets.reset()
    done = False
    
    while not done:
        random_value = random.uniform(0, 1)
        if (random_value < exploration):
            action = streets.action_space.sample() # Explore a random action
        else:
            action = np.argmax(q_table[state]) # Use the action with the highest q-value
            
        next_state, reward, done, info = streets.step(action)
        
        prev_q = q_table[state, action]
        next_max_q = np.max(q_table[next_state])
        new_q = (1 - learning_rate) * prev_q + learning_rate * (reward + discount_factor * next_max_q)
        q_table[state, action] = new_q
        
        state = next_state

A initialization of the q table is generated as a 2D Numpy array that represents every possible state and action pair in our virtual space. The parameters are specified , 10000 epochs means we would repeatedly let the taxi explore for 10000 times.

The epsilon greedy strategy comes into play when we generate a random value from the uniform distribution between 0 and 1 and compare it to our epsilon ( exploration rate ), if the random value s smaller we take a random action from our action space and if not we look at our current Q-table and take the action that maximize the value function.

We then perform the action, observe the reward and update our Q value and update our state(st) to be the “next state”(st+1).

In [7]:
q_table[initial_state]

array([-2.40301019, -2.39145865, -2.4122798 , -2.3639511 , -5.81019962,
       -7.67740217])

In [24]:
q_table[streets.encode(1,0,2,0)]  

array([-2.12208712, -2.21494995, -2.22015584, -2.21129333, -4.47957617,
       -7.05140615])

Now let’s look at the result! If we’re at the initial state, the highest value is -2.3639511, which corresponds to taking a step to the left, and if we look back to our original visualization, we can see that if the taxi wants to pick up the passenger from Yellow spot, moving to the left is obviously the best choice! You can play with this and check for different states, it is a great sanity check to see if your model converged.

In [25]:
from IPython.display import clear_output
from time import sleep
lengths=[]
for tripnum in range(1, 11):
    state = streets.reset()
   
    done = False
    trip_length = 0
    
    while not done and trip_length < 25:
        action = np.argmax(q_table[state])
        next_state, reward, done, info = streets.step(action)
        clear_output(wait=True)
        print("Trip number " + str(tripnum) + " Step " + str(trip_length))
        print(streets.render(mode='ansi'))
        sleep(.2)
        state = next_state
        trip_length += 1
    lengths.append(trip_length)
    
    sleep(.2)
avg_len=sum(lengths)/10
print(avg_len)

Trip number 10 Step 11
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

12.1


Using the Q-table we have computed we could instruct the taxi driver to do his job now. This piece of code randomly initialize a state and ask the taxi driver to perform the task according to the Q-table generated. We’ve restricted the number of steps the taxi could take to a maximum of 25 steps in case we have a taxi driver that has really bad luck and we repeat this for 10 times, calculate the average number of steps it takes for the taxi driver to complete the task to evaluate our model.

We can see that the average number of step is 12.1, not bad!

In [26]:
def q_learning(learning_rate,discount_factor,exploration,epochs):
    q_table = np.zeros([streets.observation_space.n, streets.action_space.n])
# a 2D array that represent every possible state and action in the virtual space and initialize all of them to 0
    for taxi_run in range(epochs):
        state = streets.reset()
        done = False
            
        while not done:
            random_value = random.uniform(0, 1)
            if (random_value < exploration):
                action = streets.action_space.sample() # Explore a random action
            else:
                action = np.argmax(q_table[state]) # Use the action with the highest q-value
                    
            next_state, reward, done, info = streets.step(action)
                
            prev_q = q_table[state, action]
            next_max_q = np.max(q_table[next_state])
            new_q = (1 - learning_rate) * prev_q + learning_rate * (reward + discount_factor * next_max_q)
            q_table[state, action] = new_q
                
            state = next_state



def average_trip_length():
    lengths=[]
    for tripnum in range(1, 11):
        state = streets.reset()
        done = False
        trip_length = 0
        
        while not done and trip_length < 25:
            action = np.argmax(q_table[state])
            next_state, reward, done, info = streets.step(action)
            clear_output(wait=True)
            state = next_state
            trip_length += 1
        lengths.append(trip_length)
    avg_len=sum(lengths)/10
    return avg_len

In order to improve upon our existing model, we need to tune our hyperparameters. This step is extremely important if we want our model to achieve better result, in this case, we want the taxi driver to be able to carry out the task with less step. So first I’ve re-defined the Q- learning process and calculation of average trip length functions to test different learning rate, discount factor and exploration rates (epsilon).



In [27]:
learning_rate = 0.1
discount_factor = [0.5,0.6,0.7,0.8,0.9]
exploration = 0.1
epochs = 1000
difdis=[0,0,0,0,0]
for j in range(1,10):
    for i in range(len(discount_factor)):
        q_learning(learning_rate,discount_factor[i],exploration,epochs)
        difdis[i]+=average_trip_length()

print(np.array(difdis)/10)

[12.09 12.76 13.19 12.49 12.  ]


One thing to note is that because we the initial states are randomly generated when we test for average number of steps, each time the result could be different, so we take the average of 10 trails for each average of 10 trails! Sounds confusing, but if you look closely at the code above it would become clearer. The best discount factor turns out to be `0.9`, so we fix 0.9 and try different learning rates, and do the same for exploration rates.

In [28]:
learning_rate = [0.1,0.2,0.3,0.4,0.5]
discount_factor = 0.9
exploration = 0.1
epochs = 1000
difdis=[0,0,0,0,0]
for j in range(1,10):
    for i in range(len(learning_rate)):
        q_learning(learning_rate[i],discount_factor,exploration,epochs) 
        difdis[i]+=average_trip_length()

print(np.array(difdis)/10)

[12.3  12.26 12.3  12.64 12.89]


it appears that the average trip length is the shortest or most optimal when the learning rate is set to `0.1` or 0.3.

In [29]:
learning_rate = 0.1
discount_factor = 0.9
exploration = [0.1,0.2,0.3,0.4]
epochs = 1000
difdis=[0,0,0,0]
for j in range(1,10):
    for i in range(len(exploration)):
        q_learning(learning_rate,discount_factor,exploration[i],epochs)
        difdis[i]+=average_trip_length()

print(np.array(difdis)/10)

[11.99 12.38 11.91 12.36]


it appears that the average trip length is the shortest or most optimal when the exploration is set to `0.3`.

In [30]:
learning_rate = 0.1
discount_factor = 0.9
exploration = 0.3
epochs = 1000
difdis=[]
for j in range(1,10):
        q_learning(learning_rate,discount_factor,exploration,epochs)
        difdis.append(average_trip_length())

print(sum(difdis)/10)

11.82


### Finally, combining the optimal outcomes for each parameter, we’ve achieved a much better result, the average number of steps reduced from 21.1 to 11.8!